# Dash Project Template

In [1]:
from dash import Dash, callback, html, dcc, dash_table, Input, Output, State, MATCH, ALL

In [2]:
import dash_labs as dl

In [3]:
import dash_bootstrap_components as dbc

In [4]:
from IPython.display import HTML
import os

workspace_user = os.getenv('JUPYTERHUB_USER')  # Get DS4A Workspace user name

## Goals
In previous cases, we learn the fundamentals of Dash and its essential parts, layouts, and callbacks. After this case, you will understand how to organize a complete application with Dash - Plotly.

This template uses libraries and methods to deploy a multipage and bootstrap application.

## Introduction

We will follow all our development using Jupyter lab as our script editor. To use it in the workspace, you need to add **/lab?** to the end of the URL of the DS4A workspace. If you are not using Jupyter Lab, run the following cell to get the URL for your workspace.


In [ ]:
if workspace_user:
    dash_app_url = "https://workspace.ds4a.com/user/" + workspace_user + "/lab?"

    display(
    HTML('<h2 style="background-color:Yellow;text-align:center"><a href="' + dash_app_url +
         '" target="_blank">Click here to access your Jupyter Lab!</a></h2>'))
else:    
    display(HTML('<h2 style="text-align:center"><a href="localhost:8888/lab" target="_blank">Click here to access your Jupyter Lab!</a></h2>'))

## Basic elements

To build a complete application using Dash, we need to include some concepts:

- Multipage application: A Dash web application with several pages for navigation.

- Dash Labs: Some experimental options of Dash that will be released in plotly-dash and will be available in the future as plugins.

- Bootstrap: a framework for designing and building web pages that adjust their components according to page width and some predefined styles with CSS.




## Structure

Our basic structure in the application will have some folders and files to organize our web application. Our first step is to build our folder structure.

   - app.py with our main application calls.
   - callbacks.py (main callbacks used by the application)
   - assets (this folder, will contain all custom css, javascript and images files)
   - components (customized implementations of dash core components)
   - data (sample data to interact with the application)
   - pages (navigation options available for the user)
   - requirements.txt (python dependencies)



### Exercise 1

Build the structure for the application, including the following folders

```
-- project
   |-- assets
   |-- components
       |-- kpi
       |-- maps
   |-- data
       |-- dfsample
       |-- jsonmaps
   |-- pages
 ```

---

### App.py 

App.py is our main file and the core of all our applications; however, the main goal of this file is to call out all the application functionalities; the app.py itself doesn't have all the code.

There are seven sections in this file; let's review each of them:

- **Libraries**: section all imports required for proper configuration of the dash
- **Dash instance**: method to create the Dash application
- **Top menu**: All the navigation options to deliver consistency in UX(User experience)
- **Main Layout**: Minimal container distribution and menu here with the multipage plugin; we will include our content.
- **Callback register**: A custom call to our callback.py file to insert all the callbacks of our application and to keep organized all the interactions functions in one file
- **Server variable**: This variable will be used with Gunicorn in the future to deploy our application in production mode.
- **Testing server**: Dash test server to debug until our application is ready




```python
#libraries
import dash
import dash_labs as dl
import dash_bootstrap_components as dbc
import os
#from callbacks import register_callbacks


request_path_prefix = None

#only for workspace in DS4A
workspace_user = os.getenv('JUPYTERHUB_USER')  # Get DS4A Workspace user name
if workspace_user:
    request_path_prefix = '/user/' + workspace_user + '/proxy/8050/'

    
# Dash instance declaration
app = dash.Dash(__name__, plugins=[dl.plugins.pages], requests_pathname_prefix=request_path_prefix, external_stylesheets=[dbc.themes.FLATLY],)



#Top menu, items get from all pages registered with plugin.pages
navbar = dbc.NavbarSimple([

    dbc.NavItem(dbc.NavLink( "Inicio", href=request_path_prefix)),
    dbc.DropdownMenu(
        [
            
            dbc.DropdownMenuItem(page["name"], href=request_path_prefix+page["path"])
            for page in dash.page_registry.values()
            if page["module"] != "pages.not_found_404"
        ],
        nav=True,
        label="Data Science",
    ),
    dbc.NavItem(dbc.NavLink("Nosotros", href=request_path_prefix+"/nosotros")),
    ],
    brand="DS4A Project - Team 300",
    color="primary",
    dark=True,
    className="mb-2",
)

#Main layout
app.layout = dbc.Container(
    [
        navbar,
        dl.plugins.page_container,
    ],
    className="dbc",
    fluid=True,
)

# Call to external function to register all callbacks
#register_callbacks(app)


# This call will be used with Gunicorn server
server = app.server

# Testing server, don't use in production, host
if __name__ == "__main__":
    app.run_server(host='0.0.0.0', port=8050, debug=True)
```

### Exercise 2

Create a file called **app.py** in the root folder **project** and copy the code previously presented.

----

## Pages

One of our goals is to build a multipage application; one of the reasons is to keep organized all our code, but also to prevent some performance issues trying to show too much information or cluttering the page with excessive detail.

To achieve this well-organized application, we will use, in this case, an experimental dash library called dash-labs and the plugin pages.

Each file in this folder will have a specific goal for the user interface, with their interactions and visual components.


```
-- project
   |-- pages
       |-- home.py
       |-- predictions.py
       |-- basicpage.py
```

Let's review one basic page file

- **libraries:**  All dash core and HTML components used to display on the page.
- **dash-lab plugin:** This Is the way to tell our main **app.py**, that use this page with some specific menu name and a route URL.
- **components:** Throughout our entire application, the idea is to optimize coding; sometimes, we need the same visual behaviour just with a change of information, for example, the same barplot with title and description, just with a different dataset. 
- **specific layout:** each page have its layout(distribution of graphic and interaction components, labels and HTML text)

```python
#libraries
import dash
from dash import Dash, html , dcc
import dash_bootstrap_components as dbc
from dash_labs.plugins import register_page

    
# dash-labs plugin call, menu name and route
register_page(__name__, path='/basicpage')

from components.maps.mapsample import mapsample

mapa_ejemplo = mapsample('This is my custom map', 'div_samplemap')

# specific layout for this page
layout = dbc.Container(
    [
        dbc.Row([
            dbc.Col([
                 html.H1(['Page Title'],id="div_title_maps"),
                 mapa_ejemplo.display()

            ], lg=12), 
           
        ]),
        ]
)  

```

### Exercise 3

Create a file called **basicpage.py**  **inside the pages folder** and copy the code previously presented. 

---

## Components

Dash plotly comes with many options to build the web application; these are available by importing dash-core-components and dash-bootstrap-components; you will have predefine pieces of code like dropdowns list, graph elements like maps, barplot or scatterplot.

Dash components are small pieces to build our entire application; however, we need other behaviours that suit our design and fit with the user interface.

To obtain that visual appeal, we will build some python classes based on dash components that receive our data and parameters and return s all the layout with bootstrap. 

### Component KPI - kpibadge.py

Despite our data science processes using massive datasets, we eventually need to present histograms or maps with information to aggregate some information. However, It is helpful to show some KPIs (Key Performance Indicators) that consolidate valuable information like total transactions, total users or total records for a given variable.

In this example, you find a sample layout for a class component that will receive three parameters when the instance is created.

- kip: es the number of the KPI
- label: a minimal label that shows the user the name of the KPI
- badgetype: in this case, our KPI has a colour indicator associated with the value.

```python
from dash import html 


import dash_bootstrap_components as dbc

class kpibadge:
    def __init__(self,kpi,label, badgetype):
        self.kpi = kpi
        self.label = label
        self.badgetype = badgetype
        if badgetype=='Danger':
            self.color = "danger"
        else:
             self.color = "success"

    def display(self):
        layout = html.Div(
            [
             html.Div(self.label,className='h6'),
             html.H2(self.kpi,className='d-flex justify-content-end'),
             dbc.Badge(self.badgetype, color=self.color, className="mr-1"),
            ], className='m-2'
        )
        return layout
  
```

### Exercise 4
Create a file called **kpibadge.py** **inside the components/kpi folder** and copy the code previously presented.

---

### Component Maps - mapsample.py

One of the best ways to present information is with maps, this sample uses the data available in plotly express of Montreal elections in 2013. Our class let us show an entire map inside HTML.Div with other options like a map title.

```python
from dash import html , dcc
import plotly.express as px


class mapsample:    
    """A class to represent a samplemap of Montreal Elections"""        
    def __init__(self,map_title:str,ID:str):
        """__init__
        Construct all the attributes for the sample map
     
        Args:
            map_title (str): _Title for the map_
            ID (str): _div id to specify unique #id with callbacks and css_
        
        Methods:

        display()
            Function to display a sample map with no arguments, uses plotly express data.
            
            Arguments:
                None

            Returns:
                html.Div : A Div container with a dash core component dcc.Graph() inside
        """
        
        self.map_title = map_title
        self.id = ID

    @staticmethod
    def figura():
         
        df = px.data.election() # replace with your own data source
        geojson = px.data.election_geojson()
        fig = px.choropleth(
             df, geojson=geojson, color="Bergeron",
             locations="district", featureidkey="properties.district",
             projection="mercator"                 
            )
        fig.update_geos(fitbounds="locations", visible=False)
        fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
        return fig

    def display(self):
       
        layout = html.Div(
            [
                html.H4([self.map_title]),
                html.Div([
                    dcc.Graph(figure=self.figura())
                ])
                
            ],id=self.id
        )
        return layout
```

### Exercise 5
Create a file called **mapsample.py** **inside the components/maps folder** and copy the code previously presented.

---

## Joining the parts

Now we have all the parts to build our data science web application; folder structure, app.py file with dash lab plugin to handle multipage features, a basic page layout and two unique components (kpibadge and mapsample). 

Before our final step, we will define our home page, including the previously built components and some specific python files as a good practice in our application.


### Home Page - home.py

When our user enters to the web application we will show a dashboard with several components, four kpi in the top and one map below. 

```python
import dash
from dash import html , dcc
import dash_bootstrap_components as dbc
from dash_labs.plugins import register_page


register_page(__name__, path="/")

from components.kpi.kpibadge import kpibadge
from components.maps.mapsample import mapsample


kpi1 = kpibadge('325', 'Total kpi', 'Danger')
kpi2 = kpibadge('1500', 'Total sales', 'Approved')
kpi3 = kpibadge('325', 'Total transacciones', 'Approved')
kpi4 = kpibadge('2122','Total User', 'Danger')

mapa_ejemplo = mapsample('Mapa de ejemplo', 'id_mapa_ejemplo')

layout=  dbc.Container(
    [
        dbc.Row([
            dbc.Col([
                kpi1.display()
            ], className='card'),
            dbc.Col([
                kpi2.display()
            ], className='card'),
            dbc.Col([
                kpi3.display()
            ], className='card'),
            dbc.Col([
                kpi4.display()
            ], className='card')
        ]),
        dbc.Row([
            dbc.Col([
                mapa_ejemplo.display()
            ], xs=12, className='card'),            
        ]),     
    ]
)  

```

### Excercise 6

Create a file called **home.py** **inside the pages folder** and copy the code previously presented.

---

To ensure that our components will be properly imported in each page, we need to add some final files.

### Exercise 7
Create an **empty python file** called `__init__.py` in each folder of **components** and **pages**; your file structure will look similar to this:

```
-- Project
   |-- components
       |-- __init__.py
       |-- kpi
           |-- __init__.py
           |-- kpibadge.py
       |-- maps
           |-- __init__.py
           |-- mapsample.py
   |--pages
      |-- __init__.py
```       

Finally we need to style our layout, in general we can use boostrap 5 styles, however for some specific visual details we need to add a css file with custom options.


### Exercise 8

Create a file called **styles.css** inside the folder **assets** and  copy the following styles.

```css


.card {
    #min-height:100px;
    border-width: 0;
    border-radius: 2px;
    box-shadow: 0 0 5px rgba(28, 39, 60, 0.1);
    padding: 1rem;
    margin: 0.5rem;
    margin-bottom: 1.5rem;


}
```



## Callbacks

Probably one of the most special features of Dash-plotly is the capacity to create interactions for the user in a simple way using python that render react components.

In our final file, there is a page that include a Callback, notice that you can use @callback inside this page to register the callbacks for this layout, you don't need to write callback in just one single file. 

```python
import dash
from dash_labs.plugins import register_page

register_page(__name__, path="/heatmaps")

from dash import Dash, dcc, html, Input, Output, callback
import plotly.express as px

df = px.data.medals_wide(indexed=True)

layout = html.Div(
    [
        html.P("Medals included:"),
        dcc.Dropdown(
                    id="heatmaps-medals",
                    options=[
                        {"label": "GOLD", "value": "gold"},
                        {"label": "SILVER", "value": "silver"},
                        {"label": "BRONZE", "value": "bronze"},
                    ],
                    value=['gold', 'silver', 'bronze'],
                    multi = True
                ),
        dcc.Graph(id="heatmaps-graph"),
    ], className='card'
)

@callback(
    Output("heatmaps-graph", "figure"), 
    Input("heatmaps-medals", "value"))
def filter_heatmap(cols):
    fig = px.imshow(df[cols])
    return fig

```

### Exercise 9
Create a file called **heatmaps.py** inside the **folder pages** and copy the previous code.


References:
- [Mutipage application with dash labs](https://github.com/plotly/dash-labs/blob/main/docs/demos/multi_page_example1/app.py)



### Exercise 10

Finally, let's run our web application, 

1. Open a new **terminal** in Jupyter Lab
2. Navigate to the project directory `cd <PATHDIR>` 
3. Run `python app.py`

Notes. 

- Be sure you did install the requirements dependencies with `pip install -r requeriments.txt` for this case.

- In some cases python cache files `__pycache__` can generate some errors when running the web application, if this errors shows up `ModuleNotFoundError: No module named 'pages.'` clean all cache files running this command **inside your project folder**

```shell
find . | grep -E "(.ipynb_checkpoints$|\.pyc$|\.pyo$)" | xargs rm -rf && find . | grep -E "(/__pycache__$|\.pyc$|\.pyo$)" | xargs rm -rf
```


## Takeaways

You have learned how to build an entire application with Dash-plotly. This process showed a detailed structure, you need to consider other elements that suit your needs.


- Cache optimization, some callbacks or functions can demand a lot of computing resources, if you deal with information that doesn't change to much for some graphs or tables, take a look to flask caching to improve the performance of your application. [click here for more information](https://dash.plotly.com/performance).

- Remember to use servers like Gunicorn in production, not the testing server of Dash, this server doen't handle several request in optimal way. 

- Some additional security needs to be implemented on Dash to avoid unauthorized access to your dashboard and information. A basic way to do it is to include `auth = dash_auth.BasicAuth(app,USERNAMEINFO)` [click here for more information](https://dash.plotly.com/authentication)

Additional references to improve your deployment:
- [How To Serve Flask Applications with Gunicorn and Nginx on Ubuntu 20.04](https://www.digitalocean.com/community/tutorials/how-to-serve-flask-applications-with-gunicorn-and-nginx-on-ubuntu-20-04)
